In [1]:
import numpy as np
import pandas as pd
import torch

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
device = torch.device("mps")

In [8]:
with open("../floresp-v2.0-rc.2/dev/dev.eng_Latn", encoding="UTF-8") as f:
    sentences = f.readlines()

list

In [9]:
len(sentences)

997

In [10]:
sentence = sentences[0]
inputs = tokenizer(sentence, return_tensors="pt")
model.to(device); inputs.to(device)

{'input_ids': tensor([[256047,   2153, 126376, 248079, 234283,   5057,    349, 151618,  19879,
          29470,    452, 122267, 200932,    349,   4701, 178399,    452,      9,
          10413, 131647,  70441,   1482,   2125,  24942, 149257,    811,  27888,
         248144,      9,  27594,    131,   3375,   2300,  22743,   1482,   2125,
            280,  78537,  61627,  52365,  23106,   7691,   2919,   3995,  10771,
            351,   9810,  31433,   6959,   4990,    261, 248075, 248137, 248075,
           4868,  25026, 248075, 248059,      2]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [11]:
translated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["fra_Latn"],
    num_beams=1,
    num_return_sequences=1,
    return_dict_in_generate=True,
    output_scores=True
)

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:2463: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:


In [12]:
translated_tokens.sequences.shape

torch.Size([1, 92])

In [13]:
translation = tokenizer.batch_decode(translated_tokens.sequences, skip_special_tokens=True)[0]

In [44]:
transition_scores = model.compute_transition_scores(
    translated_tokens.sequences,
    translated_tokens.scores,
    beam_indices=None,
    normalize_logits=True
).cpu()

In [45]:
transition_scores.device

device(type='cpu')

In [46]:
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = translated_tokens.sequences[:, input_length:]

probs, log_probs = [], []

output_length = np.sum(transition_scores.numpy() < 0, axis=1)

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    if not tok.item() == 1:
        probs.append(np.exp(score.numpy()))
        log_probs.append(score.numpy())
        print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

| 256057 | fra_Latn | 0.000 | 100.00%
|   155 | L        | -0.798 | 45.04%
|  7352 | undi     | -0.037 | 96.35%
| 248079 | ,        | -0.241 | 78.55%
|   702 | des      | -0.409 | 66.40%
| 69805 | scienti  | -0.125 | 88.22%
| 70342 | fiques   | -0.042 | 95.89%
|    79 | de       | -0.096 | 90.86%
|    55 | l        | -0.692 | 50.08%
| 248116 | '        | -0.090 | 91.38%
| 249117 | É        | -0.884 | 41.33%
| 46696 | cole     | -0.043 | 95.79%
|    79 | de       | -0.115 | 89.12%
| 198450 | méde     | -0.115 | 89.14%
| 39039 | cine     | -0.057 | 94.44%
|    79 | de       | -0.104 | 90.14%
|    55 | l        | -0.119 | 88.75%
| 248116 | '        | -0.089 | 91.45%
| 89559 | Univers  | -0.339 | 71.27%
|  6610 | ité      | -0.061 | 94.08%
|    79 | de       | -0.221 | 80.16%
| 151618 | Stanford | -0.105 | 90.00%
|  5303 | ont      | -0.141 | 86.87%
| 98884 | annon    | -0.064 | 93.81%
| 25535 | cé       | -0.059 | 94.24%
|    55 | l        | -0.317 | 72.81%
| 248116 | '        | -0.087 | 

In [5]:
np.exp(-0.341283289092666)

0.71085750150298

In [34]:
int(output_length[0])

In [37]:
transition_scores[0]

tensor([ 0.0000, -0.7976, -0.0372, -0.2414, -0.4094, -0.1253, -0.0419, -0.0959,
        -0.6916, -0.0901, -0.8836, -0.0431, -0.1152, -0.1149, -0.0572, -0.1038,
        -0.1194, -0.0894, -0.3387, -0.0610, -0.2212, -0.1054, -0.1408, -0.0639,
        -0.0593, -0.3174, -0.0872, -0.2447, -0.0461, -0.1210, -0.0816, -0.1098,
        -0.3098, -0.1312, -0.0577, -0.4692, -0.1293, -1.2351, -0.1233, -0.1170,
        -0.0767, -0.1849, -0.0392, -0.0315, -0.2437, -0.0746, -0.1918, -0.1610,
        -0.7081, -0.0433, -0.0096, -0.0618, -0.2125, -0.1432, -0.1926, -0.5559,
        -0.1450, -0.0928, -0.2161, -0.0787, -0.4021, -0.1741, -0.0837, -0.0442,
        -0.2319, -0.0825, -0.3176, -0.2507, -0.0760, -1.2732, -0.3144, -0.0770,
        -0.0900, -0.0053, -0.0318, -0.3595, -1.3288, -1.7838, -0.3030, -0.3689,
        -0.0732, -1.1956, -1.5971, -0.1359, -1.5946, -1.5355, -0.0952, -1.1948,
        -0.8492, -0.4507, -0.1127])

In [22]:
reconstructed_scores = transition_scores.sum(axis=1) / output_length

In [40]:
reconstructed_scores[0].item()

float

In [16]:
len(log_probs)

91

In [17]:
sequence_prob = np.prod(probs)
sequence_log_prob = np.sum(log_probs)

In [134]:
sequence_prob, sequence_log_prob

(4.0970021e-13, -28.523352)

In [95]:
model.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "max_length": 200,
  "pad_token_id": 1
}

In [71]:
np.exp(sequence_log_prob)

0.0

In [38]:
generated_tokens[0]

tensor([256057,    155,   7352, 248079,    702,  69805,  70342,     79,     55,
        248116, 249117,  46696,     79, 198450,  39039,     79,     55, 248116,
         89559,   6610,     79, 151618,   5303,  98884,  25535,     55, 248116,
            12,  36629,     13, 248116,     24,  93370,   1795,   2982,     79,
        131647, 143147,     79,   3690,     14,    591,   6293, 171388,    413,
         27888, 248144,   3335, 107806,  77947,   1350,    271,  29489,     75,
         20523,   2502,  14120,  14175, 159909, 178184,    679,     55, 248116,
         99554,     13, 248116,     75,  42808,   7272,    679,   9399,     13,
        248116,  15992,    119,  23106,   3340, 150524,    159,  86507,    338,
            79,  86507,    338,     79,     55, 248116,     24, 248075, 248059,
             2], device='mps:0')

In [62]:
translation

"Lundi, des scientifiques de l'École de médecine de l'Université de Stanford ont annoncé l'invention d'un nouvel outil de diagnostic capable de trier les cellules par type: une minuscule puce imprimable qui peut être fabriquée à l'aide d'imprimantes à jet d'encre standard à environ un centime chacun. "

In [ ]:
! python3.11 chrF++.py -R floresp-v2.0-alpha.2/dev/dev.ita_Latn -H translations/1_beam/hyp.ita_Latn

In [ ]:
round(58.8396, 1)

In [ ]:
! sacrebleu ../floresp-v2.0-rc.2/dev/dev.ukr_Cyrl -i translations/num_beams_1/hyp.ukr_Cyrl_1 -m bleu chrf ter --chrf-word-order 2

In [ ]:
! comet-score -s floresp-v2.0-alpha.2/dev/dev.eng_Latn -t translations/1_beam/hyp.arb_Arab -r floresp-v2.0-alpha.2/dev/dev.arb_Arab

## Creating Translation Jobs

In [ ]:
def generate_jobs(num_beams: int, duration_mins: int) -> None:
    """Generate jobs for translation."""

    hours, minutes = duration_mins // 60, duration_mins % 60
    time = f"{hours}:{minutes}:00"

    selected_langs = (
        pd.read_csv("../language_selection/selected_langs.csv")
        ["lang"].tolist()
    )

    for lang in selected_langs:
        with open(f"../jobs/translation/num_beams_{num_beams}/translate_{lang}_{num_beams}.sh", "w") as f:
            f.write(JOB_TEMPLATE.format(time=time, lang=lang, num_beams=num_beams))

In [48]:
JOB_TEMPLATE = """#!/bin/bash

#SBATCH --time=01:00:00
#SBATCH --partition=gpu
#SBATCH --job-name={lang}_1
#SBATCH --mem=64G
#SBATCH --gpus-per-node=1

source ~/thesis/venv/bin/activate
python ~/thesis/translate.py -t {lang} -nb 1
deactivate
"""

In [ ]:
hours, minutes = 30 // 60, 30 % 60

In [ ]:
print(f"{hours}:{minutes}:00")

In [55]:
selected_langs = (
    pd.read_csv("../language_selection/selected_langs.csv")
    ["lang"].tolist()
)

In [50]:
union_langs = (
    pd.read_csv("../language_selection/union_langs.csv")
    ["lang"].tolist()
)

In [53]:
for lang in union_langs:
    print(lang)

afr_Latn
arb_Arab
bel_Cyrl
bul_Cyrl
cat_Latn
ces_Latn
cmn_Hans
cmn_Hant
cym_Latn
dan_Latn
deu_Latn
ekk_Latn
ell_Grek
eus_Latn
fin_Latn
fra_Latn
gla_Latn
gle_Latn
glg_Latn
heb_Hebr
hin_Deva
hrv_Latn
hun_Latn
hye_Armn
ind_Latn
isl_Latn
ita_Latn
jpn_Jpan
kaz_Cyrl
kor_Hang
lit_Latn
mar_Deva
nld_Latn
pes_Arab
pol_Latn
por_Latn
ron_Latn
rus_Cyrl
san_Deva
slk_Latn
slv_Latn
spa_Latn
srp_Cyrl
swe_Latn
tam_Taml
tel_Telu
tur_Latn
uig_Arab
ukr_Cyrl
urd_Arab
vie_Latn
wol_Latn


In [57]:
for lang in selected_langs:
    with open(f"../jobs/translation/num_beams_1/translate_{lang}_1.sh", "w") as f:
        f.write(JOB_TEMPLATE.format(lang=lang))

In [ ]:
# Sort langs_to_translate by "lang"
langs_to_translate = langs_to_translate.sort_values("lang")

In [ ]:
langs_to_translate.to_csv("langs_to_translate.csv", index=False)

In [ ]:
any_feat_langs = pd.read_csv("any_feat_langs.csv")
any_feat_langs = any_feat_langs["lang"].tolist()

In [ ]:
import os

translated_langs = [
    filename[4:12]
    for filename in os.listdir("../translation/translations/num_beams_1/")
    if filename != ".DS_Store"
]

In [ ]:
scores = pd.read_csv("../translation/scores.csv")
scored_langs = scores["lang"].tolist()

In [ ]:
langs_to_score = set(translated_langs) - set(scored_langs)
langs_to_score = sorted(list(langs_to_score))

In [ ]:
for lang in langs_to_score:
    print(lang)

In [ ]:
scores

In [ ]:
scores_2 = pd.read_csv("scores_2.csv")
scores_2.drop_duplicates(subset="lang", inplace=True, ignore_index=True)

In [ ]:
scores_2_langs = scores_2["lang"].tolist()

In [ ]:
set(translated_langs) - set(scores_2_langs)

In [ ]:
scores = pd.read_csv("../translation/scores.csv")

In [ ]:
selected_langs = lang_data[lang_data["lang"].isin(scores["lang"])]

In [ ]:
selected_langs.to_csv("selected_langs.csv", index=False)

# Fucking figuring out which other languages i need to translate

In [ ]:
all_langs = pd.read_csv("../language_selection/all_langs.csv")